In [1]:
import pandas as pd

In [6]:
df=pd.read_csv(r'C:\Users\yasha\Resume Screening System\datasets\jobs_dataset_with_features.csv')

In [7]:
df.shape

(1615940, 2)

In [8]:
df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [10]:
df['Role'].value_counts()

Role
Interaction Designer            20580
Network Administrator           17470
User Interface Designer         14036
Social Media Manager            13945
User Experience Designer        13935
                                ...  
Inventory Control Specialist     3342
Budget Analyst                   3335
Clinical Nurse Manager           3324
Social Science Researcher        3321
Paid Advertising Specialist      3306
Name: count, Length: 376, dtype: int64

In [11]:
min_count=6500
role_counts=df['Role'].value_counts()
dropped_classes=role_counts[role_counts<min_count].index
filtered_df=df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)
filtered_df['Role'].value_counts()

Role
Interaction Designer          20580
Network Administrator         17470
User Interface Designer       14036
Social Media Manager          13945
User Experience Designer      13935
                              ...  
Benefits Coordinator           6839
Research Analyst               6830
Administrative Coordinator     6803
IT Support Specialist          6799
UI/UX Designer                 6743
Name: count, Length: 61, dtype: int64

In [12]:
len(filtered_df['Role'].value_counts())

61

In [13]:
df=filtered_df.sample(n=10000)

In [14]:
df.head()

,Role,Features
428747,Database Administrator,2 to 15 Years Systems Administrator BA Databas...
342506,Interaction Designer,0 to 15 Years UX/UI Designer PhD Interaction d...
274516,Supply Chain Manager,1 to 13 Years Operations Manager B.Com Supply ...
318035,Database Administrator,3 to 10 Years Systems Administrator PhD Databa...
122900,Interaction Designer,4 to 10 Years UX/UI Designer MBA Interaction d...


In [23]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

# Splitting the data into features (X) and target (y)

X = df['Features']

y = df['Role']

#Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#TF-IDF vectorization

tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [25]:
rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train_tfidf, y_train)

# Predictions

y_pred = rf_classifier.predict(X_test_tfidf)

# Accuracy

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 1.0


In [26]:
import re

def cleanResume(txt):
    cleanText = re.sub(r'http\S+\s*', ' ', txt)
    cleanText = re.sub(r'RT|cc', ' ', cleanText)
    cleanText = re.sub(r'#\S+', '', cleanText)
    cleanText = re.sub(r'@\S+', ' ', cleanText)
    cleanText = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub(r'\s+', ' ', cleanText)
    return cleanText

def job_recommendation(resume_text):

    resume_text= cleanResume(resume_text)

    resume_tfidf = tfidf_vectorizer.transform([resume_text])

    predicted_category = rf_classifier.predict(resume_tfidf)[0]

    return predicted_category

In [27]:
import pickle

pickle.dump(rf_classifier,open('rf_classifier_job_recommendation.pkl','wb'))

pickle.dump(tfidf_vectorizer,open('tfidf_vectorizer_job_recommendation.pkl','wb'))